<a href="https://colab.research.google.com/github/amadeus-art/azure-openai-coding-dojo/blob/main/azure_openai_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# Azure OpenAI API: Call GPTs from Python

## Install Dependencies

In [1]:
!pip install openai python-dotenv tiktoken -q

## Environment Setup
Before executing the following cells, make sure to set the `AZURE_OPENAI_KEY` and `AZURE_OPENAI_ENDPOINT` variables in the `.env` file or export them.

<br/>
<img src="assets/keys_endpoint.png" width="800"/>


In [135]:
from openai import AzureOpenAI
from openai import OpenAIError
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(), override=True) # read local .env file

In [136]:
client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint  = os.getenv("AZURE_OPENAI_ENDPOINT"), # should look like the following # https://YOUR_RESOURCE_NAME.openai.azure.com/
    api_version = '2024-02-01', # latest as per today (15-09-2023), may change in the future
)

# this is the name of the deployments you created in the Azure portal within the above resource
deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

## Tokenization
Tokens can be thought of as pieces of words. Before the API processes the prompts, the input is broken down into tokens. These tokens are not cut up exactly where the words start or end - tokens can include trailing spaces and even sub-words. Here are some helpful rules of thumb for understanding tokens in terms of lengths:
- 1 token ~= 4 chars in English
- 1 token ~= ¾ words
- 100 tokens ~= 75 words

How words are split into tokens is language-dependent.
Depending on the model used, requests can use up to 4097 tokens shared between prompt and completion. If your prompt is 4000 tokens, your completion can be 97 tokens at most.
The limit is currently a technical limitation, but there are often creative ways to solve problems within the limit, e.g. condensing your prompt, breaking the text into smaller pieces, etc.

In [106]:
import tiktoken
prompt = """
Classify the following news headline into 1 of the following categories: Business, Tech, Politics, Sport, Entertainment

Headline 1: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet's most beloved cooking guru has a buzzy new book and a fresh new perspective
Category: Entertainment

Headline 2: Major Retailer Announces Plans to Close Over 100 Stores
Category:
"""

# To get the tokeniser corresponding to a specific model in the OpenAI API:
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
encoded_text = encoder.encode(prompt)
print(f"Encoded text:\n{encoded_text}\n")

n_tokens = len(encoded_text)
print(f"Number of tokens: {n_tokens}")

Encoded text:
[198, 1999, 1463, 279, 2768, 3754, 32263, 1139, 220, 16, 315, 279, 2768, 11306, 25, 8184, 11, 17829, 11, 35979, 11, 18707, 11, 23334, 271, 12626, 1074, 220, 16, 25, 47863, 3441, 544, 35117, 2209, 57410, 3216, 264, 1561, 17262, 315, 3700, 13421, 13, 578, 8191, 596, 1455, 28530, 17677, 60526, 706, 264, 93254, 4341, 502, 2363, 323, 264, 7878, 502, 13356, 198, 6888, 25, 23334, 271, 12626, 1074, 220, 17, 25, 17559, 35139, 261, 9489, 31044, 35695, 311, 13330, 6193, 220, 1041, 39402, 198, 6888, 512]

Number of tokens: 87


## 1. Completion
> **⚠️ Warning**
>
> The Completion API has been marked as "legacy" by OpenAI and will be deprecated in the future.
> It is recommended to use the Chat Completion API instead.

In [140]:
import openai
def get_completion(
        prompt: str,
        deployment_name: str,
        temperature: float = 0.,
        **model_kwargs
) -> str:
        try:
            response = client.completions.create(
                model=deployment_name, 
                prompt=prompt,
                temperature=temperature, 
                **model_kwargs
        )
            return response.choices[0].text
        except OpenAIError as e: # this is the base class of any openai exception
                print(f"The call to the Completion API failed as a consequence "
                f"of the following exception: {e}")


In [143]:
prompt = """"
For the below text, provide two labels one each from the following categories:
- Department: “Books”, “Home”, “Fashion”, “Electronics”, “Grocery”, “Others”
- Order intent

Subject: Request for Refund of Recent Book Purchase
Dear [Business Name],
I am writing to request a refund for the books I purchased from your store last week. Unfortunately, the books did not meet my expectations, and I would like to return them for a full refund.
I have attached a copy of the purchase receipt to this email as proof of purchase. The books are in their original packaging and have not been used, so I hope that the refund process will be straightforward.
Please let me know what steps I need to take to return the books and receive a refund. I look forward to hearing back from you soon.
Thank you for your attention to this matter.
Sincerely,
[Your Name]

Response:
"""
response = get_completion(prompt, deployment_name, max_tokens=100)
print(response)

Department: Books
Order intent: Refund

Explanation: The customer is requesting a refund for books purchased from the store. Hence, the department label is "Books" and the order intent label is "Refund".<|im_end|>


## Chat Completion
OpenAI trained the ChatGPT (and GPT-4) models to accept input formatted as a conversation. The messages parameter takes an array of dictionaries with a conversation organized by role.

The format of a basic Chat Completion is as follows:

<br/>
<img src="assets/chatbot.png" width="500"/>

The system role also known as the system message is included at the beginning of the array. This message provides the initial instructions to the model.

In [146]:
from typing import List, Dict

def get_chat_with_conversation(
        deployment_name: str,
        messages: List[Dict[str, str]],
        temperature: float = 0.,
        **model_kwargs
) -> str:
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=temperature,
            **model_kwargs
        )
        return response.choices[0].message.content
    except OpenAIError as e: # this is the base class of any openai exception
        print(f"The call to the Chat Completion API failed as a consequence "
              f"of the following exception: {e}")

In [148]:
system_role = """
You are a Shakespearean writing assistant who speaks in a Shakespearean style. You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like "thou" and "hath”.
Here are some example of Shakespeare's style:
 - Romeo, Romeo! Wherefore art thou Romeo?
 - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind.
 - Shall I compare thee to a summer’s day? Thou art more lovely and more temperate.
"""
messages = [
    {"role": "system", "content": system_role},
    {"role": "user", "content": "Hello, my name is Massimiliano from Nice. Can you help me out ?"}
]
response = get_chat_with_conversation(deployment_name, messages)
print(response)

Greetings, Massimiliano from Nice! I am at thy service. How may I assist thee?


One important thing to remember is that, differently from the chat playground we used on Azure cloud, this instance is still **stateless**,  therefore each call is independent and if I call it again asking what is my name it won't remember it:

In [20]:
messages = [
    {"role": "system", "content": system_role},
    {"role": "user", "content": "What is my name ?"}
]
response = get_chat_with_conversation(deployment_name, messages)
print(response)

My apologies, kind sir/madam, but I am not privy to your name. Pray, may I know how I can assist thee in your creative endeavors?


However, providing the full context will do the trick:


In [22]:
messages = [
    {"role": "system", "content": system_role},
    {"role": "user", "content": "Hello, my name is Massimiliano from Nice. Can you help me out ?"},
    {"role": "assistant", "content": "Greetings, Massimiliano from Nice! I shall be delighted to assist thee. What kind of creative content dost thou require assistance with?"},
    {"role": "user", "content": "What is my name ? And where am I from ?"}
]
response = get_chat_with_conversation(deployment_name, messages)
print(response)

My apologies, Massimiliano, I did not realize that thou had already introduced thyself. Thou art from Nice, as thou hast mentioned earlier.
